# Ejercicios – Sesión 1

Instrucciones:
- No usar `show()` salvo que se indique
- Analiza antes de ejecutar
- Justifica tus respuestas


## Bloque 1 - Transformaciones vs Acciones

### Demo 1
Analizar lo que ocurre con las transformaciones y las acciones

In [0]:
#### Tu codigo ####

### Ejercicio 1 – Transformaciones vs Acciones

Observa el siguiente código:
:

```python
df2 = (
    df
    .filter(df.value > 10)
    .select("cat", "value")
    .groupBy("cat")
    .sum("value")
)
```
Sin ejecutar el código, responde:

1. ¿Se ha ejecutado Spark hasta este punto? ¿Por qué?
2. ¿Existe ya un DAG? ¿Qué contiene?
3. ¿Cuántos jobs se han lanzado?
4. ¿Qué ocurre internamente cuando ejecutamos la siguiente línea?

```python
df2.show()
```

## Bloque 2 - DAG, Stages y Tasks

### Demo - Diferentes ejemplos de creación de DAGs

In [0]:
### tu codigo ###

### Ejercicio  – Identificación de acciones

  1. Generar un DataFrame con `spark.range(0, 10000000)`.
  2. Comparar dos queries:
     - `df.groupBy("id").count()`
     - `df.filter("id % 2 = 0").count()`

  3. Preguntas:
     - ¿Cuál introduce un shuffle?
     - ¿Cómo se refleja en el `explain("extended")`?
     - ¿En cuál esperas más stages?


In [0]:
### tu codigo aqui ###

%md
### Ejercicios de DAGs respuestas rápidas

1. Count sin shuffle
```python
df = spark.range(0, 1_000_000, 1, numPartitions=8)
df.count()
```
- ¿Cuántos jobs se disparan?
- ¿Cuántos stages tendrá el job principal?
- ¿Cuántas tasks por stage?

*Tu respuesta*

2. Filtros encadenados
```python
df = spark.range(0, 1_000_000, 1, numPartitions=16)
df.filter("id > 1000").filter("id < 900000").count()
```
- ¿Qué pasa con los filtros? ¿Se ven varios en el plan físico?
- ¿Cuántos jobs/stages/tasks esperas?


*Tu respuesta*

3. Agregacion
```python
df = spark.range(0, 1_000_000, 1, numPartitions=8)
df.groupBy((df.id % 4).alias("cat")).count().show()
```
Pregunta:
- ¿Cuántos stages habrá y por qué?
- ¿Cuántas tasks esperas en el reduce? (pista: depende de spark.sql.shuffle.partitions)


*Tu respuesta*

4. Dos acciones consecutivas
```python
df = spark.range(0, 1_000_000, 1, numPartitions=4)
df.filter("id < 1000").count()
df.filter("id > 500000").show(5)
```
- ¿Cuántos jobs en total?
- ¿Se reutilizan stages entre las dos acciones?


*Tu respuesta*

### Ejercicio `explained()`
  1. Genera un DataFrame con 10 millones de filas.
  2. Aplica un `filter` y un `groupBy` con `count`.
  3. Usa `.explain("extended")` para inspeccionar el plan.
  
  ❓ Pregunta: ¿en qué momento se ejecuta la query realmente?

In [0]:
### tu codigo ###

--- 
## Soluciones


### Ejercicio 1

1. ¿Se ha ejecutado Spark hasta este punto? ¿Por qué?

👉 No, Spark no ha ejecutado nada todavía.

Todas las operaciones usadas hasta ahora son transformaciones.
Spark funciona con evaluación lazy, por lo que únicamente construye el plan lógico (DAG) sin ejecutar ningún cálculo.

2. ¿Existe ya un DAG? ¿Qué contiene?

👉 Sí, existe un DAG lógico.

El DAG contiene las siguientes transformaciones, encadenadas en este orden:

1. `filter(df.value > 10)`

2. `select("cat", "value")`

3. `groupBy("cat").sum("value")`

Este DAG describe qué hay que hacer, pero aún no se ha ejecutado.

3. ¿Cuántos jobs se han lanzado?
👉 Cero jobs.

Un job solo se lanza cuando aparece una acción.
Hasta ahora no hay ninguna acción (show, count, write, etc.)

4. ¿Qué ocurre internamente cuando ejecutamos df2.show()?

Cuando se ejecuta `df2.show()`: `show()` actúa como acción. Spark:
- Analiza el DAG
- Lo optimiza (Catalyst Optimizer)
- Lo convierte en un plan físico
- Se lanza un job
- El groupBy provoca un shuffle
- El job se divide en stages:
- Stage previo al shuffle
- Stage posterior al shuffle
- Se crean tasks, una por partición
- El resultado se envía al driver para mostrarse

### Ejercicio DAGs Codigo

In [0]:
# 1. Generar el DF
df = spark.range(0, 10_000_000)

# 2. Query A. 
df.groupBy("id").count()
df.explain("extended")
'''
- ¿Qué hace? Intenta agrupar por id.
- Como id ya es único en el rango, todos los counts = 1. Pero Spark no lo sabe antes de ejecutar.
- ¿Introduce shuffle? Sí.
- ¿Nº de stages?
➡️ Mínimo 2:
Stage 1: genera los datos (Range).
Stage 2: hace shuffle y el aggregation.
'''

# 3. Query B. 
df.filter("id % 2 = 0").count()
df.explain("extended")

'''
- ¿Qué hace? Intenta filtrar por id % 2 = 0.
- ¿Introduce shuffle? No (solo un shuffle muy pequeño al final para combinar contadores parciales).
El filter es narrow dependency: cada partición se procesa de manera independiente.
- ¿Nº de stages?
➡️ 1 o 2:
Stage 1: aplica Range + Filter.
Stage 2 (pequeño): combina los counts de cada partición.
'''

### Ejercicio DAG Corto

1. 
Respuesta esperada:
- Jobs: 1
- Stages: 1 (no hay shuffle)
- Tasks: 8 (una por partición del range)

2. 
Respuesta esperada:

- Spark fusiona los filtros en uno solo.
- Jobs: 1
- Stages: 1
- Tasks: 16 (igual que particiones iniciales).

3. 
Respuesta esperada:

- Jobs: 1
- Stages: 2 → uno map (partial agg) y uno reduce (final agg).
- Tasks: 8 en el map; en el reduce, por defecto 200 (salvo que cambies spark.sql.shuffle.partitions).

4. 
Respuesta esperada:

- Jobs: 2 (cada acción = 1 job).
- Stages: no se reutilizan → Spark vuelve a ejecutar el plan para la segunda acción.


### Ejercico `explained()`

In [0]:
# Tu código aquí 👇
df = spark.range(0, 10000000)
df2 = df.filter("id % 2 = 0").withColumn("id_squared", (df.id * df.id))
df3 = df2.groupBy((df2.id_squared % 10)).count()

df3.explain("extended")

**Solución esperada:**
- `.filter()` y `.groupBy()` son **transformaciones** → no se ejecutan inmediatamente.
- La query se ejecuta solo al llamar a una **acción** como `.count()` o `.show()`.
- En el plan físico aparece un **Exchange (shuffle)** al hacer el `groupBy`.